<p style="font-family:Times New Roman, serif; font-size:18px;text-align: justify;">
Para este proyecto estudié las opciones europeas Call y Put, desde el punto de vista del comprador y del vendedor. En el proyecto en sí, no se logra analizar todas las posibles opciones que desprende cada griega debido al trabajo del vendedor por mantener el delta neutralizado para así mantener la ganancia (prima), aislando su ganancia de la dirección del precio, además de buscar una disminución en el precio de la opción a través de Theta y Vega. Tras esto comprendí la necesidad de que el precio de la acción se mantenga constante para el vendedor.
</p>

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.stats import norm
from datetime import datetime

In [2]:
def black_scholes_call(S, K, T, r, sigma):

    #Precio de una opción de compra (Call) europea.
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = (S * norm.cdf(d1)) - (K * np.exp(-r * T) * norm.cdf(d2))
    return call_price

def black_scholes_put(S, K, T, r, sigma):

    #Calcula el precio de una opción de venta (Put) europea.
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    put_price = (K * np.exp(-r * T) * norm.cdf(-d2)) - (S * norm.cdf(-d1))
    return put_price

In [3]:
def calcular_griegas(S, K, T, r, sigma):
    
    #Función unificada para calcular d1, d2 y N'(d1) que son comunes a todas las Griegas.
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    # N'(d1) es la Función de Densidad de Probabilidad (PDF)
    N_prime_d1 = norm.pdf(d1)
    return d1, d2, N_prime_d1

def calcular_griegas_call(S, K, T, r, sigma):

    #Calcula las Griegas para una opción de compra (Call).
    d1, d2, N_prime_d1 = calcular_griegas(S, K, T, r, sigma)
    
    # Delta: N(d1)
    delta = norm.cdf(d1)
    
    # Gamma: N'(d1) / (S * sigma * sqrt(T))
    gamma = N_prime_d1 / (S * sigma * np.sqrt(T))
    
    # Vega: S * N'(d1) * sqrt(T)
    vega = S * N_prime_d1 * np.sqrt(T) * 0.01  # Por 1% de cambio en vol
    
    # Theta: -(S*N'(d1)*sigma / (2*sqrt(T))) - r*K*exp(-rT)*N(d2)
    theta_term1 = -(S * N_prime_d1 * sigma) / (2 * np.sqrt(T))
    theta_term2 = r * K * np.exp(-r * T) * norm.cdf(d2)
    theta = (theta_term1 - theta_term2) / 365  # Por 1 día
    
    # Rho: K * T * exp(-rT) * N(d2)
    rho = K * T * np.exp(-r * T) * norm.cdf(d2) * 0.01 # Por 1% de cambio en r
    
    return {'delta': delta, 'gamma': gamma, 'vega': vega, 'theta': theta, 'rho': rho}

def calcular_griegas_put(S, K, T, r, sigma):

    #Calcula las Griegas para una opción de venta (Put).
    d1, d2, N_prime_d1 = calcular_griegas(S, K, T, r, sigma)
    
    # Delta: N(d1) - 1
    delta = norm.cdf(d1) - 1
    
    # Gamma: N'(d1) / (S * sigma * sqrt(T)) (Idéntica al Call)
    gamma = N_prime_d1 / (S * sigma * np.sqrt(T))
    
    # Vega: S * N'(d1) * sqrt(T) (Idéntica al Call)
    vega = S * N_prime_d1 * np.sqrt(T) * 0.01  # Por 1% de cambio en vol
    
    # Theta: -(S*N'(d1)*sigma / (2*sqrt(T))) + r*K*exp(-rT)*N(-d2)
    theta_term1 = -(S * N_prime_d1 * sigma) / (2 * np.sqrt(T))
    theta_term2 = r * K * np.exp(-r * T) * norm.cdf(-d2)
    theta = (theta_term1 + theta_term2) / 365  # Por 1 día
    
    # Rho: -K * T * exp(-rT) * N(-d2)
    rho = -K * T * np.exp(-r * T) * norm.cdf(-d2) * 0.01 # Por 1% de cambio en r
    
    return {'delta': delta, 'gamma': gamma, 'vega': vega, 'theta': theta, 'rho': rho}

In [4]:
ticker = 'AAPL'
activo = yf.Ticker(ticker)

# S (Precio Actual del Activo) ---
S = activo.history(period='1d')['Close'].iloc[-1]
tnx_data = yf.Ticker('^TNX').history(period='5d')
r = tnx_data['Close'].iloc[-1] / 100

# Opción Real (K y T) ---
fechas_vencimiento = activo.options
print(f"Fechas de vencimiento para {ticker}: {fechas_vencimiento}")
# Elegimos una fecha de vencimiento (ej. la segunda o tercera para tener más tiempo)
fecha_vencimiento_str = fechas_vencimiento[2] 
print(f"Fecha de vencimiento elegida: {fecha_vencimiento_str}")

#  T :Tiempo hasta el Vencimiento en años
hoy = datetime.now()
fecha_vencimiento = datetime.strptime(fecha_vencimiento_str, '%Y-%m-%d')
T = (fecha_vencimiento - hoy).days / 365.25

# Cadena de opciones completa para esa fecha
opciones = activo.option_chain(fecha_vencimiento_str)
calls = opciones.calls
puts = opciones.puts 

# Elegir un K (Precio de Ejercicio) "At-the-Money" (cercano a S)
K = calls.iloc[(calls['strike'] - S).abs().argmin()]['strike']

# Sigma (Volatilidad Histórica)
hist = activo.history(period='1y')
retornos_log = np.log(hist['Close'] / hist['Close'].shift(1)).dropna()
sigma = retornos_log.std() * np.sqrt(252) 

# --- Resumen de Inputs ---
print("\n--- Inputs para el Modelo Black-Scholes ---")
print(f"Activo (S): ${S:.2f}")
print(f"Precio de Ejercicio (K): ${K:.2f}")
print(f"Tiempo al Vencimiento (T): {T:.4f} años")
print(f"Tasa Libre de Riesgo (r): {r:.4%}")
print(f"Volatilidad Histórica (sigma): {sigma:.4%}")


Fechas de vencimiento para AAPL: ('2025-11-14', '2025-11-21', '2025-11-28', '2025-12-05', '2025-12-12', '2025-12-19', '2026-01-16', '2026-02-20', '2026-03-20', '2026-04-17', '2026-05-15', '2026-06-18', '2026-08-21', '2026-09-18', '2026-12-18', '2027-01-15', '2027-06-17', '2027-12-17', '2028-01-21')
Fecha de vencimiento elegida: 2025-11-28

--- Inputs para el Modelo Black-Scholes ---
Activo (S): $268.47
Precio de Ejercicio (K): $270.00
Tiempo al Vencimiento (T): 0.0520 años
Tasa Libre de Riesgo (r): 4.0930%
Volatilidad Histórica (sigma): 32.3492%


<p style="font-family:Times New Roman, serif; font-size:18px;text-align: justify">
Una vez tenemos las variables independientes del modelo, podemos calcular el precio de las opciones call y put, en donde las compararemos con los precios que tienen en el mercado actualmente.
</p>

In [5]:
# --- Cálculo de Precios Teóricos ---
precio_call_teorico = black_scholes_call(S, K, T, r, sigma)
precio_put_teorico = black_scholes_put(S, K, T, r, sigma)

print("\n--- Precios Teóricos de las Opciones (K = ${K:.2f}) ---")
print(f"Precio Teórico del Call: ${precio_call_teorico:.2f}")
print(f"Precio Teórico del Put: ${precio_put_teorico:.2f}")

# Mercado Real (CALL)
opcion_elegida_call = calls[calls['strike'] == K].iloc[0]
precio_real_bid_call = opcion_elegida_call['bid']
precio_real_ask_call = opcion_elegida_call['ask']

print("\n--- Comparación CALL vs. Mercado Real ---")
print(f"Precio Real Call (Bid): ${precio_real_bid_call:.2f}")
print(f"Precio Real Call (Ask): ${precio_real_ask_call:.2f}")
if precio_call_teorico > precio_real_ask_call:
    print("El modelo sugiere que el CALL está SUBVALORADO (Teórico > Ask)")
elif precio_call_teorico < precio_real_bid_call:
    print("El modelo sugiere que el CALL está SOBREVALORADO (Teórico < Bid)")
else:
    print("El precio teórico del CALL está dentro del spread Bid/Ask (Precio Justo)")

# Mercado Real (PUT)
opcion_elegida_put = puts[puts['strike'] == K].iloc[0]
precio_real_bid_put = opcion_elegida_put['bid']
precio_real_ask_put = opcion_elegida_put['ask']

print("\n--- Comparación PUT vs. Mercado Real ---")
print(f"Precio Real Put (Bid): ${precio_real_bid_put:.2f}")
print(f"Precio Real Put (Ask): ${precio_real_ask_put:.2f}")
if precio_put_teorico > precio_real_ask_put:
    print("El modelo sugiere que el PUT está SUBVALORADO (Teórico > Ask)")
elif precio_put_teorico < precio_real_bid_put:
    print("El modelo sugiere que el PUT está SOBREVALORADO (Teórico < Bid)")
else:
    print("El precio teórico del PUT está dentro del spread Bid/Ask (Precio Justo)")

# Griegas (CALL) 
griegas_call = calcular_griegas_call(S, K, T, r, sigma)
print("\n--- Análisis de Riesgo (Griegas CALL) ---")
print(f"Delta (Δ): {griegas_call['delta']:.4f}")
print(f"  > Interpretación: Si {ticker} sube $1, el call subirá ${griegas_call['delta']:.2f}.")
print(f"Gamma (Γ): {griegas_call['gamma']:.4f}")
print(f"  > Interpretación: Si {ticker} sube $1, el delta subirá ${griegas_call['gamma']:.2f}.")
print(f"Vega (ν): ${griegas_call['vega']:.4f}")
print(f"  > Interpretación: Si la volatilidad sube 1%, el call subirá ${griegas_call['vega']:.2f}.")
print(f"Theta (Θ): ${griegas_call['theta']:.4f}")
print(f"  > Interpretación: Mañana, el call valdrá ${abs(griegas_call['theta']):.2f} menos.")

# Griegas (PUT) 
griegas_put = calcular_griegas_put(S, K, T, r, sigma)
print("\n--- Análisis de Riesgo (Griegas PUT) ---")
print(f"Delta (Δ): {griegas_put['delta']:.4f}")
print(f"  > Interpretación: Si {ticker} sube $1, el put PERDERÁ ${abs(griegas_put['delta']):.2f}.")
print(f"Gamma (Γ): {griegas_put['gamma']:.4f}")
print(f"  > Interpretación: Si {ticker} sube $1, el delta subirá ${griegas_put['gamma']:.2f}.")
print(f"Vega (ν): ${griegas_put['vega']:.4f}")
print(f"  > Interpretación: Si la volatilidad sube 1%, el put subirá ${griegas_put['vega']:.2f}.")
print(f"Theta (Θ): ${griegas_put['theta']:.4f}")
print(f"  > Interpretación: Mañana, el put valdrá ${abs(griegas_put['theta']):.2f} menos.")
print(f"Gamma (Γ) y Rho (ρ) también están disponibles en los diccionarios 'griegas_call' y 'griegas_put'.")


--- Precios Teóricos de las Opciones (K = ${K:.2f}) ---
Precio Teórico del Call: $7.45
Precio Teórico del Put: $8.40

--- Comparación CALL vs. Mercado Real ---
Precio Real Call (Bid): $5.10
Precio Real Call (Ask): $5.30
El modelo sugiere que el CALL está SUBVALORADO (Teórico > Ask)

--- Comparación PUT vs. Mercado Real ---
Precio Real Put (Bid): $6.25
Precio Real Put (Ask): $6.45
El modelo sugiere que el PUT está SUBVALORADO (Teórico > Ask)

--- Análisis de Riesgo (Griegas CALL) ---
Delta (Δ): 0.4955
  > Interpretación: Si AAPL sube $1, el call subirá $0.50.
Gamma (Γ): 0.0201
  > Interpretación: Si AAPL sube $1, el delta subirá $0.02.
Vega (ν): $0.2443
  > Interpretación: Si la volatilidad sube 1%, el call subirá $0.24.
Theta (Θ): $-0.2222
  > Interpretación: Mañana, el call valdrá $0.22 menos.

--- Análisis de Riesgo (Griegas PUT) ---
Delta (Δ): -0.5045
  > Interpretación: Si AAPL sube $1, el put PERDERÁ $0.50.
Gamma (Γ): 0.0201
  > Interpretación: Si AAPL sube $1, el delta subirá $0

<p style="font-family:Times New Roman, serif; font-size:18px;text-align: justify">
Podemos ver una diferencia con respecto a los precios de la opción del mercado ya sea por efectos de oferta y demanda, o por solo estar analizando una opción de forma individual.   
</p>
<p style="font-family:Times New Roman, serif; font-size:18px;text-align: justify">
Con esto busco mostrar la capacidad de obtener las distintas griegas junto a los precios teóricos de las opciones para una acción en particular (Apple).   
</p>
